### NOTEBOOK CONTENTS




In [41]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [42]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [43]:
from __future__ import print_function
from numpy import *
from scipy import *
import matplotlib.pyplot as plt
import pickle

import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import scipy as sp
import warnings
warnings.filterwarnings('ignore')

#import seawater
import datetime as dt
""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.style.use('seaborn-whitegrid')
import netCDF4 as nc

import cmocean as cm
import glob
import sys
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/CCCmaDEV/CCCma_src')
import mocsy
import CCCma
import CCCma_stations as cs
#from matplotlib import reload
import arrow
import gsw
%matplotlib inline

import xarray as xr
from datetime import datetime
from salishsea_tools import grid_tools, viz_tools

from math import log10, floor
import pytz

## Workflow - 


#### 1) For the long hindcast, extract OmA, pH, and pCO2 into daily files  > doing in pyscripts
#### 1) Extract winds?

In [44]:
#2007-sept20,2014
#y2009m08d20.nc

def make_nclist(start,end, verbose):
    
    rootdir = '/results/forcing/atmospheric/GEM2.5/'
    nc_array = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    
    
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    
    for i in range(0,len(arrow_array)):
        if i%200 == 0:
            print(i)
        tdate = arrow_array[i][0]
        ymd = tdate.format('YYYYMMDD')
        y = ymd[0:4]
        m = ymd[4:6]
        d = ymd[6:8]
        ymddate = 'y'+y+'m'+m+'d'+d
        if tdate < pytz.utc.localize(dt.datetime(2014,9,21,0,0,0)):
            t_nc = rootdir + 'gemlam/gemlam_'+ ymddate + '.nc'
        else:
            t_nc = rootdir + 'operational/ops_' + ymddate + '.nc'
        t_ncname = glob.glob(t_nc)

        if verbose == True:
            print(t_nc)
            print('file is: ' + t_ncname[0])
            print('')
        nc_array.append(t_ncname[0])
        
    return nc_array

start ='2008-01-01'
end ='2015-12-31'
verbose = False

wind_ncs = make_nclist(start,end, False)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800


In [45]:
wfile_gemlam = '/data/tjarniko/MEOPAR/grid/weights-gem2.5-gemlam_201702.nc'
wfile_ops = '/data/tjarniko/MEOPAR/grid/weights-gem2.5-ops_201702.nc'

def weights_intrp_mtx(odata,wfile):
    '''POINT: interpolates data from the atmospheric grid onto the ocean grid using a weights matrix | CALLS: | NOTES: this is much faster than interpolating with scipy! | TAKES : one array of size lats 266 x lons 256 of atmospheric data | GIVES: an array of NEMO-Salish size of the new data interpolated | USAGE: | ROLE IN CLUSTER PROJECT: | written by Michael Dunphy, adapted by tjšj, 2017'''
    from IPython import embed
    import netCDF4 as nc
    import scipy.interpolate as spi
    import scipy.sparse as sp
    import numpy as np
    '''takes a matrix of size lon(256), lat(266) and interpolates it onnto the NEMO gird
    size is 266 x 256'''
    
    with nc.Dataset(wfile) as f:
        s1 = f.variables['src01'][:]-1  # minus one for fortran-to-python indexing
        s2 = f.variables['src02'][:]-1
        s3 = f.variables['src03'][:]-1
        s4 = f.variables['src04'][:]-1
        w1 = f.variables['wgt01'][:]
        w2 = f.variables['wgt02'][:]
        w3 = f.variables['wgt03'][:]
        w4 = f.variables['wgt04'][:]
       
    NO = odata.size   # number of operational grid points
    NN = s1.size      # number of NEMO grid points
    
    # Build matrix
    n = np.array([x for x in range(0,NN)])
    M1 = sp.csr_matrix((w1.flatten(), (n, s1.flatten())), (NN,NO))
    M2 = sp.csr_matrix((w2.flatten(), (n, s2.flatten())), (NN,NO))
    M3 = sp.csr_matrix((w3.flatten(), (n, s3.flatten())), (NN,NO))
    M4 = sp.csr_matrix((w4.flatten(), (n, s4.flatten())), (NN,NO))
    M = M1+M2+M3+M4
    
        # Interpolate by matrix multiply - quite fast
    ndata = M*odata.flatten()

    # Reshape to NEMO shaped array
    ndata=ndata.reshape(s1.shape)
    
    return ndata

In [46]:
w = nc.Dataset(wind_ncs[0])
u_wind = w['u_wind'][:]
v_wind = w['v_wind'][:]
wind_mag = np.sqrt((u_wind)*(u_wind)+(v_wind)*(v_wind))
daily_wm = np.nanmean(wind_mag, axis=0)

daily_wm_interp = weights_intrp_mtx(daily_wm,wfile_gemlam)

In [47]:
def make_windlist(start,end):
    
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    
    
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    
    for i in range(0,len(arrow_array)):

        tdate = arrow_array[i][0]
        ymd = tdate.format('YYYYMMDD')
        if i%50 == 0:
            print(ymd)
        y = ymd[0:4]
        m = ymd[4:6]
        d = ymd[6:8]
        ymddate = 'y'+y+'m'+m+'d'+d
        
        #decide which wind weights file to use
        if tdate < pytz.utc.localize(dt.datetime(2014,9,21,0,0,0)):
            wfile =  '/data/tjarniko/MEOPAR/grid/weights-gem2.5-gemlam_201702.nc'

        else:
            wfile = '/data/tjarniko/MEOPAR/grid/weights-gem2.5-ops_201702.nc'
            
        w = nc.Dataset(wind_ncs[i])
        u_wind = w['u_wind'][:]
        v_wind = w['v_wind'][:]
        wind_mag = np.sqrt((u_wind)*(u_wind)+(v_wind)*(v_wind))
        daily_wm = np.nanmean(wind_mag, axis=0)

        daily_wm_interp = weights_intrp_mtx(daily_wm,wfile_gemlam)

        ymd = tdate.format('YYYYMMDD')
        ncname = '/data/tjarniko/results/hindcast.201905_windmag_interp/windmag_interp_1d_' + ymd + '.nc'
        
        f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
        g = f.createGroup('model_output')
        g.createDimension('ydir',898)
        g.createDimension('xdir',398)

        ts = g.createVariable('wind_mag','f4',('ydir','xdir'))
        ts[:] = daily_wm_interp
        f.close()



In [48]:
start ='2008-01-01'
end ='2015-12-31'
make_windlist(start,end)

20080101
20080220
20080410
20080530
20080719
20080907
20081027
20081216
20090204
20090326
20090515
20090704
20090823
20091012
20091201
20100120
20100311
20100430
20100619
20100808
20100927
20101116
20110105
20110224
20110415
20110604
20110724
20110912
20111101
20111221
20120209
20120330
20120519
20120708
20120827
20121016
20121205
20130124
20130315
20130504
20130623
20130812
20131001
20131120
20140109
20140228
20140419
20140608
20140728
20140916
20141105
20141225
20150213
20150404
20150524
20150713
20150901
20151021
20151210
